In [ ]:
import os

print(os.getcwd())

In [ ]:
import pandas as pd

best_algorithms = pd.read_csv('Best algorithms.csv')

In [ ]:
# Add in maxmin for LA1 to match the others
import glob
import math
import numpy as np
import re
LA1_model_output_list = glob.glob("/Documents/LA1 Results July 2020/Models/model_output_*.csv")

LA1_maxmin_df = pd.DataFrame(data = {'model_id': ['rq1_ts_all', 'rq1_ss_str', 'rq2_ss_all', 'rq2_ts_str',
       'rq1_ts_str', 'rq1_ss_all', 'rq2_ss_str', 'rq2_ts_all'],
                                    'Algorithm': ['Maximum lower bound (training)'] * 8,
                                    'LA': ['LA1'] * 8})

LA1_model_output_dict = {}
for file_name in LA1_model_output_list:
    file = pd.read_csv(file_name)
    file_name = file_name.replace('/Documents/LA1 Results July 2020/Models/model_output_', '')
    file_name = file_name.replace('.csv', '')
    if 'ts' in file_name:
        n_splits = 3
    else:
        n_splits = 5
    # 
    file.reset_index(drop= True, inplace = True)
    max_min_mean_idx = np.argmax(file['mean_test_score'] - 1.96* file['std_test_score']/math.sqrt(n_splits))
    max_min_mean = round(file.loc[max_min_mean_idx, 'mean_test_score'], 2)
    model_id = re.findall('rq[12]_[a-z]{2}_[a-z]{2,3}', file_name)[0]
    algorithm = file_name.split(model_id+'_')[1]
    LA1_maxmin_df.at[LA1_maxmin_df['model_id'] == model_id, algorithm] = max_min_mean

In [ ]:
# Pull in list of model performance metrics
import glob
import pandas as pd
file_list_excel = glob.glob("*/*.xlsx")
file_list_csv = glob.glob("*/*.csv")
file_list = file_list_excel + file_list_csv

file_list = [f for f in file_list if 'scores' in f]
file_list = [f for f in file_list if 'Best algorithms' not in f]

file_dict = {}
for file_name in file_list:
    if 'LA1' in file_name:
        # LA1
        file = pd.read_excel(file_name, header = 1)
         
    else:
        # All other LAs' scores are in csv form
        file = pd.read_csv(file_name, skiprows = 1)
    file_name = file_name.replace("../../Summary information FINAL/", "")        
    if 'multiple' in file_name:
        file_name = file_name.replace("scores_multiple_", "")    

    else:
        file_name = file_name.replace("scores_", "")
    file_name = file_name.replace(".xlsx", "")
    file_name = file_name.replace(".csv", "")
    LA, model_id = file_name.split('/')
    file['LA'] = LA
    file['model_id'] = model_id
    file_dict[file_name] = file


print(file_dict.keys())

results = pd.concat(file_dict.values(), axis = 0, ignore_index = True)

In [ ]:
LA1_maxmin_df.to_csv('LA1/LA1 maxmimum lower bound.csv')

In [ ]:
results.loc[results['LA'] == 'LA4','Algorithm'].value_counts()

In [ ]:
## Merge in LA1 maxmin
print(results.shape)
results = pd.concat([results, LA1_maxmin_df], axis = 0)
print(results.shape)

'''
results.loc[(results['LA'] == 'LA1') &
    (results['Algorithm'].isin(['Maximum lower bound (training)', 'Mean average precision (training)'])),].sort_values(by = 'model_id')
'''

In [ ]:
# Rename Algorithm
results = results.rename(columns = {'Algorithm': 'Metrics'})

# Separate out model_id to allow for easier subsetting of the data
model_id = results['model_id'].str.split('_', expand = True)
results['rq'], results['cv'], results['data_type'] = model_id[0], model_id[1], model_id[2]

In [ ]:
# Merge in which was the best algorithm
import numpy as np
results = results.merge(best_algorithms, how='left', left_on = ['LA', 'model_id'], right_on=['LA', 'Model'])
results.drop(columns = 'model_id', inplace = True)

# Identify the value of the metric for the algorithm we chose
results['Best algorithm model performance'] = np.where(results['Best algorithm'] == 'decision_tree',
                                                        results['decision_tree'], 
                                                       np.where(results['Best algorithm'] == 'logistic_regression',
                                                       results['logistic_regression'], results['gradient_boosting']))

## Cross validation

In [ ]:
# Create dataframe comparing cv results
results_cv = pd.pivot_table(results, index=['Metrics', 'LA', 'rq', 'data_type'], columns = 'cv', values='Best algorithm model performance')
results_cv.reset_index(inplace = True)
results_cv['Validation technique with best model performance'] = np.where((results_cv['ss']>results_cv['ts']) &
                                                                          (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                       'Learning from all cases', 
                                                       np.where((results_cv['ss']<results_cv['ts']) &
                                                                          (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                             'Learning only from earlier cases',   
                                                        np.where((results_cv['ss']==results_cv['ts']) &
                                                                          (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                         'Same performance', np.nan)))

results_cv['Validation technique with best model performance'] = np.where((results_cv['ss']<results_cv['ts']) &
                                                           (results_cv['Metrics']=='Std average precision (training)'),
                                                             'Learning from all cases',
                                                                    np.where((results_cv['ss']>results_cv['ts']) &
                                                           (results_cv['Metrics']=='Std average precision (training)'),
                                                             'Learning only from earlier cases', results_cv['Validation technique with best model performance']))
    

In [ ]:
# Better than threshold?
results_cv['Greater than "success" threshold (0.65) - learning from all cases'] = np.where((results_cv['ss'] >= 0.65) &
                                                        (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                                 'Exceeds success threshold',
                                                                 np.where((results_cv['ss'] < 0.65) &
                                                        (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                                         'Does not exceeds success threshold' ,np.nan))
                                                                                                   
results_cv['Greater than "success" threshold (0.65) - Learning only from earlier cases'] = np.where((results_cv['ts'] >= 0.65) &
                                                        (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                                 'Exceeds success threshold',
                                                                 np.where((results_cv['ts'] < 0.65) &
                                                        (results_cv['Metrics']!='Proportion negative class value') &
                                                           (results_cv['Metrics']!='Std average precision (training)'),
                                                                         'Does not exceeds success threshold' ,np.nan))                                                                 
                                                                 

In [ ]:
# Relabel
results_cv.rename(columns = {'LA': 'Local authority',
                        'rq': 'Prediction',
                        'ss': 'Learning from all cases',
                        'ts': 'Learning only from earlier cases',
                         'data_type': 'Data included'},
              inplace = True)

# Renaming before saving
print(results_cv['Data included'].value_counts())
results_cv['Data included'] = np.where(results_cv['Data included']=='str',
                                  'Structured data only',
                                   'Structured and text data')
print(results_cv['Data included'].value_counts())

In [ ]:
# Renaming predictions according to the LA number and the rq number 
results_cv['Prediction Number'] = np.where(((results_cv['Local authority']=='LA1') & 
                                  (results_cv['Prediction'] == 'rq1')),
                                1,
                                np.where(((results_cv['Local authority']=='LA1') & 
                                  (results_cv['Prediction'] == 'rq2')),
                                2,
                                np.where(((results_cv['Local authority']=='LA2') & 
                                  (results_cv['Prediction'] == 'rq1')),
                                3,
                                np.where(((results_cv['Local authority']=='LA2') & 
                                  (results_cv['Prediction'] == 'rq2')),
                                4,
                                np.where(((results_cv['Local authority']=='LA3') & 
                                  (results_cv['Prediction'] == 'rq1')),
                                5,
                                np.where(((results_cv['Local authority']=='LA3') & 
                                  (results_cv['Prediction'] == 'rq2')),
                                6,
                                np.where(((results_cv['Local authority']=='LA4') & 
                                  (results_cv['Prediction'] == 'rq1')),
                                7,8)))))))

results_cv.drop(columns = ['Prediction'], inplace = True)


## Data type

In [ ]:
# Create dataframe comparing data_type results
results_data_type = pd.pivot_table(results, index=['Metrics', 'LA', 'rq', 'cv'], columns = 'data_type', values='Best algorithm model performance')
results_data_type.reset_index(inplace = True)

results_data_type['Data included with best model performance'] = np.where((results_data_type['all']>results_data_type['str']) &
                                                                          (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                       'Structured and text data', 
                                                       np.where((results_data_type['all']<results_data_type['str']) &
                                                                          (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                             'Structured data only',   
                                                        np.where((results_data_type['all']==results_data_type['str']) &
                                                                          (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                         'Same performance', np.nan)))

results_data_type['Data included with best model performance'] = np.where((results_data_type['all']<results_data_type['str']) &
                                                           (results_data_type['Metrics']=='Std average precision (training)'),
                                                             'Structured and text data',
                                                                    np.where((results_data_type['all']>results_data_type['str']) &
                                                           (results_data_type['Metrics']=='Std average precision (training)'),
                                                             'Structured data only', results_data_type['Data included with best model performance']))
    


In [ ]:
# Better than threshold?
results_data_type['Greater than "success" threshold (0.65) - structured data only'] = np.where((results_data_type['str'] >= 0.65) &
                                                        (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                                 'Exceeds success threshold',
                                                                 np.where((results_data_type['str'] < 0.65) &
                                                        (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                                         'Does not exceeds success threshold' ,np.nan))
                                                                                                   
results_data_type['Greater than "success" threshold (0.65) - structured and text data'] = np.where((results_data_type['all'] >= 0.65) &
                                                        (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                                 'Exceeds success threshold',
                                                                 np.where((results_data_type['all'] < 0.65) &
                                                        (results_data_type['Metrics']!='Proportion negative class value') &
                                                           (results_data_type['Metrics']!='Std average precision (training)'),
                                                                         'Does not exceeds success threshold' ,np.nan))    

                                               

In [ ]:
# Dropping unnecessary columns and remaning columns
results_data_type.rename(columns = {'LA': 'Local authority',
                        'rq': 'Prediction', 'cv': 'Cross Validation Method',
                         'all': 'Structured and text data',
                        'str': 'Structured data only'},
              inplace = True)

# Renaming before saving
print(results_data_type['Cross Validation Method'].value_counts())
results_data_type['Cross Validation Method'] = np.where(results_data_type['Cross Validation Method']=='ss',
                                             'Learning from all cases',
                                             'Learning only from earlier cases')
print(results_data_type['Cross Validation Method'].value_counts())


In [ ]:
# Renaming predictions according to the LA number and the rq number 
results_data_type['Prediction Number'] = np.where(((results_data_type['Local authority']=='LA1') & 
                                  (results_data_type['Prediction'] == 'rq1')),
                                1,
                                np.where(((results_data_type['Local authority']=='LA1') & 
                                  (results_data_type['Prediction'] == 'rq2')),
                                2,
                                np.where(((results_data_type['Local authority']=='LA2') & 
                                  (results_data_type['Prediction'] == 'rq1')),
                                3,
                                np.where(((results_data_type['Local authority']=='LA2') & 
                                  (results_data_type['Prediction'] == 'rq2')),
                                4,
                                np.where(((results_data_type['Local authority']=='LA3') & 
                                  (results_data_type['Prediction'] == 'rq1')),
                                5,
                                np.where(((results_data_type['Local authority']=='LA3') & 
                                  (results_data_type['Prediction'] == 'rq2')),
                                6,
                                np.where(((results_data_type['Local authority']=='LA4') & 
                                  (results_data_type['Prediction'] == 'rq1')),
                                7,8)))))))

results_data_type.drop(columns = ['Prediction'], inplace = True, errors='ignore')
results_data_type = results_data_type[['Metrics', 'Local authority', 'Cross Validation Method',
        'Structured data only', 'Structured and text data',
       'Data included with best model performance', 
        'Greater than "success" threshold (0.65) - structured data only',
       'Greater than "success" threshold (0.65) - structured and text data',
        'Prediction Number']]

In [ ]:
# Split by prediction and save into the right files
# Save average precision and mean and sd on training

for pred in results_cv['Prediction Number'].unique():
    LA = results_cv.loc[results_cv['Prediction Number'] == pred, 'Local authority'].reset_index(drop=True)[0]
    
    # Cross validation
    results_cv_pred = results_cv.loc[
                    ((results_cv['Metrics'] == 'Proportion negative class value') & 
                     (results_cv['Prediction Number'] == pred)) |
                    ((results_cv['Metrics'] == 'Accuracy') & 
                    (results_cv['Prediction Number'] == pred)) |
                    ((results_cv['Metrics'] == 'Average precision') & 
                   (results_cv['Prediction Number'] == pred)) |
                   ((results_cv['Metrics'] == 'AUC') &
                   (results_cv['Prediction Number'] == pred)) |
                   ((results_cv['Metrics'] == 'Mean average precision (training)') &
                   (results_cv['Prediction Number'] == pred)) |
                ((results_cv['Metrics'] == 'Std average precision (training)') &
                   (results_cv['Prediction Number'] == pred)),]
    
    results_cv_pred.drop(columns = ['Local authority', 'Prediction Number'], inplace = True)
    results_cv_pred['Metrics'] = pd.Categorical(results_cv_pred['Metrics'], 
                                            categories = ['Proportion negative class value',
                                                          'Accuracy',
                                                        'Average precision', 
                                                          'Mean average precision (training)',
                                                          'Std average precision (training)',
                                                            'AUC'], 
                                            ordered = True)
    results_cv_pred.sort_values(by = ['Metrics', 'Data included'], inplace = True)

    results_cv_pred.to_csv('{}/Model summary cv {}.csv'.format(LA, pred), index = False)
    
    # Data type
    results_data_type_pred = results_data_type.loc[
                    ((results_data_type['Metrics'] == 'Proportion negative class value') & 
                    (results_data_type['Prediction Number'] == pred)) |
                    ((results_data_type['Metrics'] == 'Accuracy') & 
                   (results_data_type['Prediction Number'] == pred)) |
                    ((results_data_type['Metrics'] == 'Average precision') & 
                   (results_data_type['Prediction Number'] == pred)) |
                    ((results_data_type['Metrics'] == 'AUC') &
                    (results_data_type['Prediction Number'] == pred)) |
                   ((results_data_type['Metrics'] == 'Mean average precision (training)') &
                   (results_data_type['Prediction Number'] == pred)) |
                    ((results_data_type['Metrics'] == 'Std average precision (training)') &
                   (results_data_type['Prediction Number'] == pred)),]
    
    results_data_type_pred.drop(columns = ['Local authority', 'Prediction Number'], inplace = True)
    results_data_type_pred['Metrics'] = pd.Categorical(results_data_type_pred['Metrics'], 
                                            categories = ['Proportion negative class value',
                                                          'Accuracy',
                                                          'Average precision', 
                                                          'Mean average precision (training)',
                                                          'Std average precision (training)',
                                                            'AUC'], 
                                            ordered = True)
    results_data_type_pred.sort_values(by = ['Metrics', 'Cross Validation Method'], inplace = True)
    results_data_type_pred.to_csv('{}/Model summary data type {}.csv'.format(LA, pred), index = False)

In [ ]:
# Don't include accuracy here
metrics_to_report = ['Average precision', 'Mean average precision (training)', 'AUC']

for metric in metrics_to_report:
    metrics_beat_threshold_summary_ss = pd.crosstab(results_cv.loc[results_cv['Metrics'] == metric, 'Data included'], 
                                    results_cv.loc[results_cv['Metrics'] == metric, 
                                    'Greater than "success" threshold (0.65) - learning from all cases'], margins = True)
    metrics_beat_threshold_summary_ts = pd.crosstab(results_cv.loc[results_cv['Metrics'] == metric, 'Data included'], 
                                    results_cv.loc[results_cv['Metrics'] == metric, 
                                    'Greater than "success" threshold (0.65) - Learning only from earlier cases'], margins = True)
    print(metrics_beat_threshold_summary_ss)
    print(metrics_beat_threshold_summary_ts)

In [ ]:
metrics_beat_threshold_summary_ss

In [ ]:
metrics_beat_threshold_summary_ts

In [ ]:
# Dropping unnecessary columns and remaning columns
results.drop(columns = ['decision_tree', 'gradient_boosting', 'logistic_regression', 'Model'], inplace = True)
results.rename(columns = {'LA': 'Local authority',
                        'rq': 'Prediction', 'cv': 'Cross Validation Method',
                         'data_type': 'Data included'},
              inplace = True)

In [ ]:
# Renaming before saving
print(results['Data included'].value_counts())
results['Data included'] = np.where(results['Data included']=='str',
                                  'Structured data only',
                                   'Structured and text data')
print(results['Data included'].value_counts())

print(results['Cross Validation Method'].value_counts())
results['Cross Validation Method'] = np.where(results['Cross Validation Method']=='ss',
                                             'Learning from all cases',
                                             'Learning only from earlier cases')
print(results['Cross Validation Method'].value_counts())

results['Best algorithm'] = np.where(results['Best algorithm']=='gradient_boosting',
                                    'Gradient Boosting',
                                    np.where(results['Best algorithm']=='decision_tree',
                                    'Decision Tree',
                                            'Logistic Regression'))

In [ ]:
# Renaming predictions according to the LA number and the rq number 
results['Prediction Number'] = np.where(((results['Local authority']=='LA1') & 
                                  (results['Prediction'] == 'rq1')),
                                1,
                                np.where(((results['Local authority']=='LA1') & 
                                  (results['Prediction'] == 'rq2')),
                                2,
                                np.where(((results['Local authority']=='LA2') & 
                                  (results['Prediction'] == 'rq1')),
                                3,
                                np.where(((results['Local authority']=='LA2') & 
                                  (results['Prediction'] == 'rq2')),
                                4,
                                np.where(((results['Local authority']=='LA3') & 
                                  (results['Prediction'] == 'rq1')),
                                5,
                                np.where(((results['Local authority']=='LA3') & 
                                  (results['Prediction'] == 'rq2')),
                                6,
                                np.where(((results['Local authority']=='LA4') & 
                                  (results['Prediction'] == 'rq1')),
                                7,8)))))))

results.drop(columns = 'Prediction', inplace = True)


In [ ]:
# Exceeds threshold?

results['Greater than "success" threshold (0.65)'] = np.where((results['Best algorithm model performance'] >= 0.65) &
                                                        (results['Metrics']!='Proportion negative class value') &
                                                           (results['Metrics']!='Std average precision (training)'),
                                                                 'Exceeds success threshold',
                                                                 np.where((results['Best algorithm model performance'] < 0.65) &
                                                        (results['Metrics']!='Proportion negative class value') &
                                                           (results['Metrics']!='Std average precision (training)'),
                                                                         'Does not exceeds success threshold' ,np.nan))


In [ ]:
#results = results.loc[results['Local authority']!='LA1',] # For the visualisation

In [ ]:
mask = ((results['Metrics']!='Proportion negative class value') &
       (results['Metrics']!='Std average precision (training)'))

metrics_exceed_threshold = pd.DataFrame(pd.crosstab(results.loc[mask, 'Metrics'], 
            results.loc[mask, 'Greater than "success" threshold (0.65)']))

metrics_exceed_threshold.to_csv('Output/Exceeds threshold by metric.csv')

In [ ]:
metrics_exceed_threshold

In [ ]:
# Get name
question_names = pd.read_csv("Years and Sample Sizes.csv")
question_names['Prediction Number'] = np.where((question_names['Local authority'] == 'LA1')
                                               & (question_names['Research question'] == 'rq1'),1,
                                        np.where((question_names['Local authority'] == 'LA1')
                                               & (question_names['Research question'] == 'rq2'), 2,
                                        np.where((question_names['Local authority'] == 'LA2')
                                               & (question_names['Research question'] == 'rq1'),3,
                                        np.where((question_names['Local authority'] == 'LA2')
                                               & (question_names['Research question'] == 'rq2'), 4, 
                                        np.where((question_names['Local authority'] == 'LA3')
                                               & (question_names['Research question'] == 'rq1'),5,
                                        np.where((question_names['Local authority'] == 'LA3')
                                               & (question_names['Research question'] == 'rq2'), 6,
                                         np.where((question_names['Local authority'] == 'LA4')
                                               & (question_names['Research question'] == 'rq1'),7, 8)))))))  
results = results.merge(question_names[['Local authority', 'Research question', 'Shortened outcome', 'Prediction Number']], 
                                                  how = 'left', left_on = ['Local authority', 'Prediction Number'], right_on = ['Local authority', 'Prediction Number'])

In [ ]:
results_threshold = results.loc[results['Metrics'].isin(['AUC', 'Average precision']), ]
results_threshold = results_threshold[['Local authority','Research question',
       'Prediction Number', 'Shortened outcome', 'Metrics',
       'Cross Validation Method', 'Data included',
       'Best algorithm model performance',
       'Greater than "success" threshold (0.65)']]
print(results_threshold.shape)
results_threshold.drop_duplicates(inplace = True)
print(results_threshold.shape)

In [ ]:
results_threshold.head()

In [ ]:
results_threshold.to_csv('Meet threshold.csv', index = False)

In [ ]:
results_comparison_metrics = results.loc[results['Metrics'].isin(['Accuracy', 'AUC', 'Average precision']), ]
results_comparison_metrics = results_comparison_metrics[['Local authority','Research question',
       'Prediction Number', 'Shortened outcome', 'Metrics',
       'Cross Validation Method', 'Data included',
       'Best algorithm model performance']]
print(results_comparison_metrics.shape)
results_comparison_metrics.drop_duplicates(inplace = True)
print(results_comparison_metrics.shape)
results_comparison_metrics.head()

In [ ]:
results_comparison_metrics.to_csv('Comparison of metrics.csv', index = False)

In [ ]:
# Reorder columns
# Threshold visualisation
results = results[['Local authority',
 'Prediction Number',
 'Cross Validation Method',
 'Data included',
 'Metrics',
  'Best algorithm',
 'Best algorithm model performance']]
# Sort values
results.sort_values(by=['Local authority', 'Prediction Number', 'Cross Validation Method', 'Data included', 'Metrics'], inplace = True)
# Save
results.to_csv('Output/Summary table.csv')

In [ ]:
results.head()

In [ ]:
# Find the difference between the AP in validation and the AP on holdout data
results_diff_validation_holdout = results.loc[(results['Metrics'] == 'Maximum lower bound (training)') | 
                                             (results['Metrics'] == 'Average precision'), ]
print(results_diff_validation_holdout.shape)
results_diff_validation_holdout.sort_values(by = ['Local authority', 'Prediction Number', 'Cross Validation Method',
       'Data included', 'Metrics'], inplace = True)
results_diff_validation_holdout = results_diff_validation_holdout.groupby(['Local authority', 'Prediction Number', 'Cross Validation Method',
       'Data included']).diff()

# Merge back in to allow for groupbys
results_diff_validation_holdout.rename(columns = {'Best algorithm model performance': 
                                        'Difference between average precision in validation and on holdout data'},
                                      inplace = True)

results_diff_validation_holdout = results.merge(results_diff_validation_holdout, left_index = True, right_index = True, how = 'right')
results_diff_validation_holdout.dropna(axis = 0, how = 'any', subset = ['Difference between average precision in validation and on holdout data'], inplace = True)



In [ ]:
results_diff_validation_holdout['Difference between average precision in validation and on holdout data'].mean()

In [ ]:
# Calculate average drop of AP from validation to holdout by algorithm, data included and cv method

overfitting_by_algorithm = pd.DataFrame(results_diff_validation_holdout.groupby('Best algorithm')['Difference between average precision in validation and on holdout data'].mean())
overfitting_by_data_type = pd.DataFrame(results_diff_validation_holdout.groupby('Data included')['Difference between average precision in validation and on holdout data'].mean())
overfitting_by_cv = pd.DataFrame(results_diff_validation_holdout.groupby('Cross Validation Method')['Difference between average precision in validation and on holdout data'].mean())

overfitting_by_algorithm['Comparison'] = 'Algorithm'
overfitting_by_algorithm.reset_index(drop = False, inplace = True)
overfitting_by_algorithm.rename(columns = {'Best algorithm': 'Difference averaged by:'}, inplace = True)

overfitting_by_data_type['Comparison'] = 'Data included'
overfitting_by_data_type.reset_index(drop = False, inplace = True)
overfitting_by_data_type.rename(columns = {'Data included': 'Difference averaged by:'}, inplace = True)

overfitting_by_cv['Comparison'] = 'Cross Validation Method'
overfitting_by_cv.reset_index(drop = False, inplace = True)
overfitting_by_cv.rename(columns = {'Cross Validation Method': 'Difference averaged by:'}, inplace = True)

overfitting = pd.concat([overfitting_by_algorithm, overfitting_by_data_type, overfitting_by_cv], axis = 0)

In [ ]:
# Reorder columns, round and save

overfitting = overfitting[['Comparison', 'Difference averaged by:', 'Difference between average precision in validation and on holdout data']]
overfitting['Difference between average precision in validation and on holdout data'] = round(overfitting['Difference between average precision in validation and on holdout data'], 2)
overfitting.to_csv('Output/Overfitting by group.csv', index = False)

In [ ]:
overfitting

In [ ]:
#assert 1==2

In [ ]:
# Comparison of metrics
results.head()

In [ ]:
# Count of which cv method is best
best_cv = pd.DataFrame(results_cv.loc[results_cv['Metrics']=='Average precision','Validation technique with best model performance'].value_counts())
best_cv.reset_index(inplace = True)
best_cv.rename(columns = {'index': 'Cross validation technique', 'Validation technique with best model performance':'Number of times the technique is the best performing'}, inplace = True)

best_cv.to_csv('Output/Best cross validation table.csv')


In [ ]:
best_cv

In [ ]:
# Count of which data_type method is best
best_data_type = pd.DataFrame(results_data_type.loc[results_cv['Metrics']=='Average precision','Data included with best model performance'].value_counts())
best_data_type.reset_index(inplace = True)
best_data_type.rename(columns = {'index': 'Data included in model', 'Data included with best model performance':'Number of times the model with this data included is the best performing'}, inplace = True)

best_data_type.to_csv('Output/Best data type table.csv')

In [ ]:
best_data_type

In [ ]:
# Count of which cv method is best
best_algorithm = pd.DataFrame(results.loc[results['Metrics']=='Average precision','Best algorithm'].value_counts())
best_algorithm.reset_index(inplace = True)
best_algorithm.rename(columns = {'index': 'Algorithm', 'Best algorithm':'Number of times the algorithm is the best performing'}, inplace = True)

best_algorithm.to_csv('Output/Best algorithm table.csv')

In [ ]:
best_algorithm

In [ ]:
# Rank sum test - are the performance metrics better for ss?
# Include all of the metrics where > is better. The number of average precision scores is too low
# Not prespecified
from scipy.stats import mannwhitneyu

results_mw = results_cv.loc[(results_cv['Metrics']!='Proportion negative class value') &
                           (results_cv['Metrics']!='Std average precision (training)')]
ss = results_mw['Learning from all cases']
ts = results_mw['Learning only from earlier cases']

mw_cv = mannwhitneyu(ss, ts, alternative='two-sided')
print(mw_cv)

In [ ]:
# Rank sum test - are the performance metrics better for all data_type?
# Include all of the metrics where > is better. The number of average precision scores is too low
# Not prespecified

from scipy.stats import mannwhitneyu

results_mw = results_data_type.loc[(results_data_type['Metrics']!='Proportion negative class value') &
                           (results_data_type['Metrics']!='Std average precision (training)')]
structured = results_mw['Structured data only']
structured_text = results_mw['Structured and text data']

mw_data_type = mannwhitneyu(structured, structured_text, alternative='two-sided')
print(mw_data_type)

In [ ]:
mw_df = pd.DataFrame(data = {'Mann Whitney U Statistic': [mw_cv[0], mw_data_type[0]],
                        'P-value': [mw_cv[1], mw_data_type[1]]}, 
                        index=['Cross-Validation technique', 'Data included in model'])
mw_df.to_csv('Output/Mann Whitney tests.csv')

In [ ]:
# Calculate overall mean and std of average precision
results_mean = results.loc[results['Metrics'] == 'Average precision','Best algorithm model performance'].mean()
results_std = results.loc[results['Metrics'] == 'Average precision','Best algorithm model performance'].std()
print(results_mean)
print(results_std)

In [ ]:
# Calculate overall mean and std of average precision excluding LA1
results_excl_LA1 = results.loc[results['Local authority'] !='LA1',]
results_excl_LA1_mean = results_excl_LA1.loc[results_excl_LA1['Metrics'] == 'Average precision','Best algorithm model performance'].mean()
results_excl_LA1_std = results_excl_LA1.loc[results_excl_LA1['Metrics'] == 'Average precision','Best algorithm model performance'].std()
print(results_excl_LA1_mean)
print(results_excl_LA1_std)

In [ ]:
overall_summary = pd.DataFrame(data = {'Mean': [results_mean, results_excl_LA1_mean],
                                      'Standard deviation': [results_std, results_excl_LA1_std]},
                              index = ['Local authorities 1-4', 'Local authorities 2-4'])

In [ ]:
overall_summary.to_csv('Output/Means and standard deviations.csv')

In [ ]:
overall_summary

## Compare 4 models

In [ ]:
# Build pivot table to compare across the 4 models
results_pivoted = pd.pivot_table(results, index=['Metrics', 'Local authority', 'Prediction Number'], columns = ['Cross Validation Method', 'Data included'], values='Best algorithm model performance')

# Just include scores where higher is better
print(results_pivoted.shape)
metrics_higher_is_better = [r for r in results_pivoted.index if 'Proportion negative class value' not in r and 'Std average precision (training)' not in r]
results_pivoted = results_pivoted.loc[metrics_higher_is_better,]
print(results_pivoted.shape)
results_pivoted

In [ ]:
# ts_all vs ts_str
results_pivoted['Comparison of data included for models learning from all cases'] = np.where((results_pivoted[('Learning only from earlier cases', 'Structured and text data')]>results_pivoted[('Learning only from earlier cases', 'Structured data only')]),
                                                       'Learning only from earlier cases, Structured and text data', 
                                            np.where((results_pivoted[('Learning only from earlier cases', 'Structured and text data')]<results_pivoted[('Learning only from earlier cases', 'Structured data only')]),
                                                             'Learning only from earlier cases, Just structured data',
                                                        np.where((results_pivoted[('Learning only from earlier cases', 'Structured and text data')]==results_pivoted[('Learning only from earlier cases', 'Structured data only')]),
                                               'Same performance', np.nan)))   

# ss_all vs ss_str
results_pivoted['Comparison of data included for models learning only from earlier cases'] = np.where((results_pivoted[('Learning from all cases', 'Structured and text data')]>results_pivoted[('Learning from all cases', 'Structured data only')]),
                                                       'learning from all cases, Structured and text data', 
                                            np.where((results_pivoted[('Learning from all cases', 'Structured and text data')]<results_pivoted[('Learning from all cases', 'Structured data only')]),
                                                             'learning from all cases, Just structured data',
                                                        np.where((results_pivoted[('Learning from all cases', 'Structured and text data')]==results_pivoted[('Learning from all cases', 'Structured data only')]),
                                               'Same performance', np.nan)))


In [ ]:
# ss_str vs ts_str
results_pivoted['Comparison of cross validation for just structured data models'] = np.where((results_pivoted[('Learning only from earlier cases', 'Structured data only')]>results_pivoted[('Learning from all cases', 'Structured data only')]),
                                                       'Learning only from earlier cases, Just structured data', 
                                            np.where((results_pivoted[('Learning only from earlier cases', 'Structured data only')]<results_pivoted[('Learning from all cases', 'Structured data only')]),
                                                             'learning from all cases, Just structured data',
                                                        np.where((results_pivoted[('Learning only from earlier cases', 'Structured data only')]==results_pivoted[('Learning from all cases', 'Structured data only')]),
                                               'Same performance', np.nan)))   

# ts_all vs ss_all
results_pivoted['Comparison of cross validation for structured and text data models'] = np.where((results_pivoted[('Learning only from earlier cases', 'Structured and text data')]>results_pivoted[('Learning from all cases', 'Structured and text data')]),
                                                       'Learning only from earlier cases, Structured and text data', 
                                            np.where((results_pivoted[('Learning only from earlier cases', 'Structured and text data')]<results_pivoted[('Learning from all cases', 'Structured and text data')]),
                                                             'learning from all cases, Structured and text data',
                                                        np.where((results_pivoted[('Learning only from earlier cases', 'Structured and text data')]==results_pivoted[('Learning from all cases', 'Structured and text data')]),
                                               'Same performance', np.nan)))  

In [ ]:
data_included_time_aware = pd.DataFrame(results_pivoted.loc[[r for r in results_pivoted.index if 'Average precision' in r], 'Comparison of data included for models learning from all cases'].value_counts())
data_included_time_aware.to_csv('Output/Comparison of data included for models learning from all cases.csv')

data_included_non_time_aware = pd.DataFrame(results_pivoted.loc[[r for r in results_pivoted.index if 'Average precision' in r], 'Comparison of data included for models learning only from earlier cases'].value_counts()) 
data_included_non_time_aware.to_csv('Output/Comparison of data included for models learning only from earlier cases.csv')


cross_validation_structured = pd.DataFrame(results_pivoted.loc[[r for r in results_pivoted.index if 'Average precision' in r], 'Comparison of cross validation for just structured data models'].value_counts())
cross_validation_structured.to_csv('Output/Comparison of cross validation for just structured data models.csv')

cross_validation_structured_text = pd.DataFrame(results_pivoted.loc[[r for r in results_pivoted.index if 'Average precision' in r], 'Comparison of cross validation for structured and text data models'].value_counts())
cross_validation_structured_text.to_csv('Output/Comparison of cross validation for structured and text data models.csv')


In [ ]:
results_pivoted.to_csv('Output/Summary table - comparison of cross validation and data included.csv')

In [ ]:
#assert 1==2

## Create values for visualisations

In [ ]:
results_pivoted

In [ ]:
results_pivoted.columns


In [ ]:

results_for_visualisation = results_pivoted.drop(columns = ['Comparison of data included for models learning from all cases', 
'Comparison of data included for models learning only from earlier cases',
'Comparison of cross validation for just structured data models',
'Comparison of cross validation for structured and text data models'
])
comparison_of_metrics = results_for_visualisation.iloc[(results_for_visualisation.index.get_level_values('Metrics') == 'Accuracy') |
                              (results_for_visualisation.index.get_level_values('Metrics') == 'AUC') |
                              (results_for_visualisation.index.get_level_values('Metrics') == 'Average precision'), ]




In [ ]:
results_for_visualisation.iloc[(results_for_visualisation.index.get_level_values('Metrics') == 'Precision') |
                              (results_for_visualisation.index.get_level_values('Metrics') == 'Recall'),]

In [ ]:
false_alarms_children_missed = results_for_visualisation.iloc[(results_for_visualisation.index.get_level_values('Metrics') == 'Precision') |
                              (results_for_visualisation.index.get_level_values('Metrics') == 'Recall'),]


false_alarms_children_missed[('Learning from all cases','Structured and text data')] = (1 - false_alarms_children_missed[('Learning from all cases','Structured and text data')]) * 100
false_alarms_children_missed[('Learning from all cases','Structured data only')] = (1 - false_alarms_children_missed[('Learning from all cases','Structured data only')]) * 100
false_alarms_children_missed[('Learning only from earlier cases','Structured and text data')] = (1 - false_alarms_children_missed[('Learning only from earlier cases','Structured and text data')]) * 100
false_alarms_children_missed[('Learning only from earlier cases','Structured data only')] = (1 - false_alarms_children_missed[('Learning only from earlier cases','Structured data only')]) * 100




In [ ]:
# Add shortened name for predicted outcome
false_alarms_children_missed.reset_index(inplace = True)
false_alarms_children_missed['Metrics'] = false_alarms_children_missed['Metrics'].replace({'Precision': 'False alarms', 'Recall': 'Children at risk missed'})
false_alarms_children_missed


In [ ]:
# Import shortened prediction questions

question_names = pd.read_csv("Years and Sample Sizes.csv")
question_names['Prediction Number'] = np.where((question_names['Local authority'] == 'LA1')
                                               & (question_names['Research question'] == 'rq1'),1,
                                        np.where((question_names['Local authority'] == 'LA1')
                                               & (question_names['Research question'] == 'rq2'), 2,
                                        np.where((question_names['Local authority'] == 'LA2')
                                               & (question_names['Research question'] == 'rq1'),3,
                                        np.where((question_names['Local authority'] == 'LA2')
                                               & (question_names['Research question'] == 'rq2'), 4, 
                                        np.where((question_names['Local authority'] == 'LA3')
                                               & (question_names['Research question'] == 'rq1'),5,
                                        np.where((question_names['Local authority'] == 'LA3')
                                               & (question_names['Research question'] == 'rq2'), 6,
                                         np.where((question_names['Local authority'] == 'LA4')
                                               & (question_names['Research question'] == 'rq1'),7, 8)))))))    
                    
                                               

In [ ]:
false_alarms_children_missed_summary = false_alarms_children_missed.merge(question_names[['Prediction Number', 'Shortened outcome']], on = 'Prediction Number', how = 'left')
false_alarms_children_missed_summary.head()

In [ ]:
# Merge in the shortened outcome names
false_alarms_children_missed_summary = false_alarms_children_missed_summary[[
                                    ('Local authority', ''), 'Prediction Number', 
                                    'Shortened outcome',
                                     ('Metrics', ''),
                                     ('Learning from all cases', 'Structured and text data'),
                    ('Learning from all cases', 'Structured data only'),
       ('Learning only from earlier cases', 'Structured and text data'),
           ('Learning only from earlier cases', 'Structured data only')]]
false_alarms_children_missed_summary.sort_values(by = [('Local authority', ''), 'Prediction Number'], inplace = True)

In [ ]:
false_alarms_children_missed_summary.head()

In [ ]:
# Then check all are available and match with what's in the techincal report
false_alarms_children_missed_summary.drop_duplicates(inplace = True)
false_alarms_children_missed_summary[('Metrics','')].value_counts()

In [ ]:
false_alarms_children_missed_summary.to_csv('False alarms and children at risk.csv', index = False)

## Learning curve data

In [ ]:
import pandas as pd

learning_curve_list = glob.glob("/Documents/Summary information FINAL/*/Learning curve data *.csv")


learning_curve_dict = {}
for file_name in learning_curve_list:
    file = pd.read_csv(file_name)
    file_name = file_name.replace('/Documents/Summary information FINAL/', '')
    file_name = file_name.replace('.csv', '')
    learning_curve_dict[file_name] = file

In [ ]:
learning_curve_dict.keys()

In [ ]:
learning_curve_data = pd.concat(learning_curve_dict, axis = 0)
learning_curve_data.reset_index(drop = False, inplace = True)
learning_curve_data.rename(columns = {'level_0': 'Graph file name'}, inplace = True)

In [ ]:
learning_curve_data['Cross validation'] = np.where(learning_curve_data['model_id'].str.contains('ss'),
                                             'Learning from all cases',
                                             'Learning only from earlier cases')

learning_curve_data['Data included'] = np.where(learning_curve_data['model_id'].str.contains('str'),
                                            'Structured data only',
                                           'Structured and text data')

In [ ]:
# Renaming model_ids according to the LA number and the rq number 
learning_curve_data['Prediction Number'] = np.where(((learning_curve_data['LA']=='LA1') & 
                                  (learning_curve_data['model_id'].str.contains('rq1'))),
                                1,
                                np.where(((learning_curve_data['LA']=='LA1') & 
                                  (learning_curve_data['model_id'].str.contains( 'rq2'))),
                                2,
                                np.where(((learning_curve_data['LA']=='LA2') & 
                                  (learning_curve_data['model_id'].str.contains('rq1'))),
                                3,
                                np.where(((learning_curve_data['LA']=='LA2') & 
                                  (learning_curve_data['model_id'].str.contains( 'rq2'))),
                                4,
                                np.where(((learning_curve_data['LA']=='LA3') & 
                                  (learning_curve_data['model_id'].str.contains('rq1'))),
                                5,
                                np.where(((learning_curve_data['LA']=='LA3') & 
                                  (learning_curve_data['model_id'].str.contains( 'rq2'))),
                                6,
                                np.where(((learning_curve_data['LA']=='LA4') & 
                                  (learning_curve_data['model_id'].str.contains('rq1'))),
                                7,8)))))))

In [ ]:
learning_curve_data.head()

In [ ]:
learning_curve_data = learning_curve_data[['Number of observations',
       'Average precision - train', 'Average precision - test', 'LA',
       'Cross validation', 'Data included', 'Prediction Number', 'Graph file name']]

In [ ]:
learning_curve_data.head()

In [ ]:
learning_curve_data_LA4 = pd.read_csv('/Final/Would more data help 7 and 8.csv')
learning_curve_data_LA4['Graph file name'] = 'LA4/' + learning_curve_data_LA4['Graph file name']
learning_curve_data_LA4.head()

In [ ]:
learning_curve_data_all = pd.concat([learning_curve_data, learning_curve_data_LA4], axis = 0)
learning_curve_data_all.shape

In [ ]:
learning_curve_data_all['Prediction Number'].value_counts()

In [ ]:
learning_curve_data_all.to_csv('Final/Take Three/Would more data help all.csv', index = False)